---
# Model T - Transfer Learning, Feature Extraction, RMSProp
- **128 x 128** x 3 Image size.  
- **64** Batch size.  
- **VGG16** Convolutional Base.
- **Feature Extraction**.  
- **Root Mean Square Propagation (RMSProp)** optimizer.
- **4 x 4 x 512** Tensor before flatten.  
- **Classifier**: 
    - 512 Dense layer with **ReLU** activation function and **L1L2** regularization.
    - Root Mean Square Propagation **(RMSProp) optimizer** with 0.001 Learning Rate.  
    - **30 Epochs** to train the classifier.  
- Build the **full model** and test it.

---
#### Imports and Setup

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
print(f'TensorFlow version: {tf.__version__}')
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(3)
import matplotlib.pyplot as plt
import pickle
import numpy as np
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow import keras
from tensorflow.keras import callbacks, layers, optimizers, models
from keras import regularizers
print(f"Keras Version: {keras.__version__}")
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay ,accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc
from sklearn.preprocessing import label_binarize
from itertools import cycle

- We resize the images to **128 x 128** pixels.  
- With this image size, we expect a **4 x 4 x 512** feature map after the convolutional base.  

---
#### Group Datasets

In [ ]:
IMG_SIZE = 128

train_dirs = [f'../data/train1_resized_{IMG_SIZE}', f'../data/train3_resized_{IMG_SIZE}', f'../data/train4_resized_{IMG_SIZE}', f'../data/train5_resized_{IMG_SIZE}']
validation_dir = f'../data/train2_resized_{IMG_SIZE}'
test_dir = f'../data/test_resized_{IMG_SIZE}'

- ((2221985 + 2221986) % 5) + 1 = 2  
- **Validation set: train2**.  

---
#### Create Datasets

In [ ]:
BATCH_SIZE = 64
NUM_CLASSES = 10

train_datasets = [image_dataset_from_directory(directory, image_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE) for directory in train_dirs]

train_dataset = train_datasets[0]
for dataset in train_datasets[1:]:
    train_dataset = train_dataset.concatenate(dataset)

train_dataset = train_dataset.shuffle(buffer_size=1000).prefetch(buffer_size=tf.data.AUTOTUNE)
validation_dataset = image_dataset_from_directory(validation_dir, image_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = image_dataset_from_directory(test_dir, image_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

class_names = train_datasets[0].class_names

for data_batch, labels_batch in train_dataset.take(1):
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)

- We define the batch size of 64 and create an array with the label's names.  
- We create the train dataset by concatenating them, we **shuffle** the samples before each epoch and **prefetch** them to memory.  
- We do the same for the validation and test dataset except shuffling which is unnecessary.

---
#### Loading the VGG16 Model

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
conv_base.trainable = False
conv_base.summary()

- We load the VGG16 model with the imagenet weights, without the top layer and with the input shape of 128 x 128 pixels and 3 channels.

---
#### Feature Extraction

In [ ]:
def get_features_and_labels(dataset):
    all_features = []
    all_labels = []
    for images, labels in dataset:
        preprocessed_images = keras.applications.vgg16.preprocess_input(images)
        features = conv_base.predict(preprocessed_images)
        all_features.append(features)
        all_labels.append(labels)
    return np.concatenate(all_features), np.concatenate(all_labels)

train_features, train_labels = get_features_and_labels(train_dataset)
val_features, val_labels = get_features_and_labels(validation_dataset)

- We extract the features from the convolutional base of the VGG16 model for the train and validation dataset.
- We don't need to extract the features from the test dataset because we will use the full model to predict it.

---
### Saving the features and labels

In [ ]:
np.save('../features/04_model_t_feat_ext_rmsprop_train_features.npy', train_features)
np.save('../features/04_model_t_feat_ext_rmsprop_train_labels.npy', train_labels)
np.save('../features/04_model_t_feat_ext_rmsprop_val_features.npy', val_features)
np.save('../features/04_model_t_feat_ext_rmsprop_val_labels.npy', val_labels)

---
### Loading the features and labels

In [ ]:
train_features = np.load('../features/04_model_t_feat_ext_rmsprop_train_features.npy')
train_labels = np.load('../features/04_model_t_feat_ext_rmsprop_train_labels.npy')
val_features = np.load('../features/04_model_t_feat_ext_rmsprop_val_features.npy')
val_labels = np.load('../features/04_model_t_feat_ext_rmsprop_val_labels.npy')

---
#### Classifier Arquitecture

In [ ]:
inputs = keras.Input(shape=(4, 4, 512))
x = layers.Flatten()(inputs)
x = layers.Dropout(0.5)(x)
x = layers.Dense(512, activation="relu", kernel_regularizer=regularizers.L2(1e-4))(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax", kernel_regularizer=regularizers.L2(1e-4))(x)
classifier = models.Model(inputs=inputs, outputs=outputs)
classifier.summary()

- Input size of 4 x 4 x 512.  
- Flatten the tensor to a 8192 x 1 tensor. 
- A 512 and 10 dense output layer, one for each category. 
- Dropout of 0.5 before the dense layers.
- L1 regularization 0.0001 and L2 regularization 0.001 on the dense layers.    
- ReLU activation function on the dense layer.  
- Softmax activation function on the output layer for multiclass classification.  

---
#### Model Compilation

In [ ]:
initial_learning_rate = 0.001
optimizer = optimizers.RMSprop(learning_rate=initial_learning_rate)
loss_function = keras.losses.SparseCategoricalCrossentropy()

lr_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=1)
save_best_model = callbacks.ModelCheckpoint(filepath='../models/04_model_t_feat_ext_rmsprop_classifier.keras', save_best_only=True, monitor='val_loss', verbose=1)

callbacks = [lr_scheduler, early_stopping, save_best_model]

classifier.compile(optimizer=optimizer,
              loss=loss_function,
              metrics=['accuracy'])

- **RMSProp** as the optimizer for this model with an initial learning rate of 0.001.  
- **Sparse categorical cross entropy** as the loss function, because our labels are not one hot encoded.  
- **Learning rate scheduler** to lower the learning rate by a factor of 0.1 on validation loss plateau (patience of 2).  
- **Early train stopping** based on validation loss improvement (stops when validation loss doesn't improve for 4 straight epochs (patience of 4)).  
- **Checkpoints** to save the best model between each epoch based on validation loss.

---
#### Model Training
- We train the classifier with the extracted features during 30 epochs.

In [ ]:
history = classifier.fit(train_features,
                    train_labels,
                    epochs=30,
                    validation_data=(val_features, val_labels),
                    callbacks=callbacks)

---
#### Save Model History

In [ ]:
with open("../history/04_model_t_feat_ext_rmsprop_classifier.pkl", "wb") as file:
    pickle.dump(history.history, file)

---
#### Classifier Evaluation

In [ ]:
val_loss, val_acc = classifier.evaluate(val_features, val_labels)
print(f'Classifier Validation Loss: {val_loss:.2f}')
print(f'Classifier Validation Accuracy: {val_acc:.2%}')

---
#### Model Training Visualization

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.tight_layout()
plt.show()

- Analyzing the training and validation accuracy and loss over the epochs:
    - We see that the model begins slightly overfitting after the **15th** epoch. The validation accuracy stops improving significantly while the training accuracy keeps improving.  
    - The validation loss stops improving significantly after the **15th** epoch while the training loss keeps improving. 
    - The best model, based on validation loss, is saved on the **20th** epoch.   

---
#### Building the full model (VGG16 + Classifier)

In [ ]:
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = keras.applications.vgg16.preprocess_input(inputs)
x = conv_base(x)
outputs = classifier(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    loss=loss_function,
    optimizer=optimizer,
    metrics=["accuracy"])

model.summary()
model.save('../models/04_model_t_feat_ext_rmsprop_full_model.keras')

- We join the VGG16 convolutional base with our classifier trained with the extracted features, compile it, and save it.

---
#### Model Testing

In [ ]:
test_labels = []
test_predictions = []
test_probabilities = []

for images, labels in test_dataset:
    test_labels.extend(labels.numpy())
    predictions = model.predict(images)
    test_predictions.extend(np.argmax(predictions, axis=-1))
    test_probabilities.extend(predictions)

test_labels = np.array(test_labels)
test_predictions = np.array(test_predictions)
test_probabilities = np.array(test_probabilities)

---
#### Confusion Matrix

In [ ]:
cm = confusion_matrix(test_labels, test_predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues, xticks_rotation=90)
plt.show()

- Looking at the confusion matrix, we see that the model has some trouble distinguishing between some categories.  
- The model has a hard time distinguishing between the categories 003_cat and 005_dog.  
- The model also has a hard time distinguishing between some other categories but the error is not as significant.  
- The model has an acceptable performance on the categories 000_airplane, 001_automobile, 004_deer, 006_frog, 007_horse, 008_ship and 009_truck.

---
#### ROC Curve Analysis

In [ ]:
test_labels_bin = label_binarize(test_labels, classes=range(NUM_CLASSES))

false_positive_rate = dict()
true_positive_rate = dict()
roc_auc = dict()

for i in range(NUM_CLASSES):
    false_positive_rate[i], true_positive_rate[i], _ = roc_curve(test_labels_bin[:, i], test_probabilities[:, i])
    roc_auc[i] = auc(false_positive_rate[i], true_positive_rate[i])

plt.figure(figsize=(10, 8))
colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'blue', 'green', 'red', 'purple', 'brown', 'pink', 'grey'])
for i, color in zip(range(NUM_CLASSES), colors):
    plt.plot(false_positive_rate[i], true_positive_rate[i], color=color, lw=2, label=f'Class {class_names[i]} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

- We see that the model has a good performance on the ROC curve for most categories.  
- The categories 003_cat, 003_bird, 005_dog and 004_deer have the worst AUC (Area Under Curve) performance.
- A perfect AUC of 1.0 would mean that the model classifies all true positives and true negatives correctly.

---
#### Performance Metrics
- **Accuracy** is the proportion of correctly predicted instances out of the total instances.  
- **Precision** is the ratio of true positive predictions to the total predicted positives. Macro precision calculates this for each class independently and then averages them.  
- **Weighted precision** calculates the precision for each class, then averages them, weighted by the number of true instances for each class.  
- **Recall** is the ratio of true positive predictions to the total actual positives. Macro recall calculates this for each class independently and then averages them.  
- **Weighted recall** calculates the recall for each class, then averages them, weighted by the number of true instances for each class.  
- The **F1-score** is the harmonic mean of precision and recall. Macro F1-score calculates this for each class independently and then averages them.  
- **Weighted F1-score** calculates the F1-score for each class, then averages them, weighted by the number of true instances for each class.  

In [ ]:
acc = accuracy_score(y_true =  test_labels, y_pred = test_predictions)
print(f'Accuracy : {np.round(acc*100,2)}%')
precision = precision_score(y_true =  test_labels, y_pred = test_predictions, average='macro')
print(f'Precision - Macro: {np.round(precision*100,2)}%')
recall = recall_score(y_true =  test_labels, y_pred = test_predictions, average='macro')
print(f'Recall - Macro: {np.round(recall*100,2)}%')
f1 = f1_score(y_true =  test_labels, y_pred = test_predictions, average='macro')
print(f'F1-score - Macro: {np.round(f1*100,2)}%')
precision = precision_score(y_true =  test_labels, y_pred = test_predictions, average='weighted')
print(f'Precision - Weighted: {np.round(precision*100,2)}%')
recall = recall_score(y_true =  test_labels, y_pred = test_predictions, average='weighted')
print(f'Recall - Weighted: {np.round(recall*100,2)}%')
f1 = f1_score(y_true =  test_labels, y_pred = test_predictions, average='weighted')
print(f'F1-score - Weighted: {np.round(f1*100,2)}%')

- **Since the dataset is balanced, the **MACRO** average is a good metric to evaluate the model.**

# Conclusion
- We resized our images to be the 128 x 128.
- We extracted feature maps from our datasets using the convolutional base of the VGG16.
- We have trained a classifier with those extracted features.  
- We experimented with various classifier architectures, and we settled for this one, keeping input size to 128 x 128 and our classifier with a 512 dense layer.  
     - Different learning rates were tested; we settled for the Reduce Learning Rate on Plateau callback with a starting value of 0.001.
    - Various batch sizes were explored; this size was optimal for this architecture.  
    - Multiple optimizers were evaluated; we settled for RMSProp.
    - Several regularization values were tried; these values worked best.
    - Different dropout rates were assessed; this rate provided the best results.
    - Various epoch counts were tested; 30 epochs were optimal.
    - We used L1 and L2 regularization on the classifier.
    - Dropout was applied to the input layer and the dense layer.
- The model showed some difficulty distinguishing between certain categories, particularly cats and dogs.
- Overfitting was observed after **15 epochs**, but the best model was saved at the **20th epoch**.
- We evaluated the model using a confusion matrix to analyze its performance on each category.
- We evaluated the model using ROC curves for a deeper performance analysis.
- **The model achieved an accuracy of 81.83% on the test set**.
- Performance on the test set was good, with:
    - Macro F1-score: 89.29%
    - Weighted F1-score: 89.29%
    - Macro precision: 89.32%
    - Weighted precision: 89.32%
    - Macro recall: 89.29%
    - Weighted recall: 89.29%

### Future Work
- In the next phase, we will:
    - Implement and test transfer learning with the VGG16 convolutional base and our classifier along with fine-tuning by later unfreezing some of the VGG16 layers to further improve model generalization.
    - Experiment with data augmentation to improve model generalization.
    - Explore additional regularization methods to address overfitting.